In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
# Matching dictionary for team names between events and tracking data
TEAM_NAMES = {'Canada': 'Olympic (Women) - Canada',
              'USA': 'Olympic (Women) - United States',
              'Finland': 'Olympic (Women) - Finland',
              'ROC': 'Olympic (Women) - Olympic Athletes from Russia',
              'Switzerland': 'Olympic (Women) - Switzerland'}

In [3]:
tracking_dir = "external/Big-Data-Cup-2022/data"
pbp_dir = "external/Big-Data-Cup-2022/data"
play_by_play_data_file = 'pxp_womens_oly_2022_v2.csv'
power_play_info_file = 'pp_info.csv'

In [4]:
play_by_play_data = pd.read_csv(os.path.join(tracking_dir, play_by_play_data_file))

In [5]:
def sort_row_by_col(row, col="player_name"):
    
    try:
        if row[f"{col}_2"] < row[col]:
            for col in ["player_name", "jn", "position"]:
            # for col in ["player_name"]:
                tmp = row[f"{col}_2"]
                row[f"{col}_2"] = row[col]
                row[col] = tmp
    except:
        print(row)
            
    return row

In [6]:
def run(game, game_date, team_name):
    this_game = play_by_play_data.copy()
    # power_play_info = pd.read_csv(os.path.join(pbp_dir, power_play_info_file))
    roster_info = pd.read_csv(os.path.join(tracking_dir, game, f"{game} roster.csv"), index_col=0)

    # Merge with roster data of current game
    this_game = this_game[this_game["game_date"] == game_date]
    this_game = this_game[this_game["team_name"] == team_name]
    this_game = this_game[(this_game["event"] == "Play") & this_game["event_successful"]]
    this_game = this_game.join(roster_info, on = "player_name").join(roster_info, on = "player_name_2", rsuffix="_2")
    # Exclude goalie
    # this_game = this_game[(this_game["position"] != "Goalie") & (this_game["position_2"] != "Goalie")]

    # By player
    # collected_passes = this_game.apply(lambda row: sort_row_by_col(row, col="player_name"), axis=1)
    # collected_passes = collected_passes.loc[:, ["player_name", "player_name_2"]].value_counts()
    # for period in [1, 2, 3]:
        # collected_passes = this_game[this_game["period"] == period]
    
    # By position
    collected_passes = this_game.apply(lambda row: sort_row_by_col(row, col="position"), axis=1)
    # Remove loops
    loops = collected_passes[collected_passes['position'] == collected_passes['position_2']]
    print(f"#Loops: {len(loops)}")
    collected_passes = collected_passes.loc[:, ["position", "position_2"]].value_counts()
    passes_by_players = this_game.apply(lambda row: sort_row_by_col(row, col="player_name"), axis=1)
    passes_by_players.loc[:, ["position", "position_2"]].value_counts()
    print(collected_passes)
    
    # Write to CSV
    collected_passes.to_csv(f"{game} passes.csv")
    return loops, passes_by_players

In [7]:
game = "2022-02-14 Switzerland at Canada"
game_date = "14/2/2022"
team_name = TEAM_NAMES["Canada"] 
loops, passes_by_players = run(game, game_date, team_name)

#Loops: 13
position       position_2   
Center         Right Wing       53
Left Defense   Left Wing        51
Center         Right Defense    44
Left Wing      Right Wing       44
Left Defense   Right Defense    42
Center         Left Wing        38
               Left Defense     36
Right Defense  Right Wing       36
Left Defense   Right Wing       32
Left Wing      Right Defense    28
               RightWing        16
Right Defense  RightWing        14
Defense        Left Wing        13
Center         RightWing        12
               Defense          11
Left Defense   RightWing         8
Defense        Right Wing        7
Left Defense   Left Defense      6
Goalie         Right Defense     6
Defense        RightWing         6
               Right Defense     5
Right Defense  Right Defense     4
Left Wing      Left Wing         2
Defense        Left Defense      2
Goalie         Left Defense      2
Center         Center            1
Defense        Goalie            1
Center         

In [9]:
loops = loops.apply(lambda row: sort_row_by_col(row, col="player_name"), axis=1)
loops.loc[:, ["player_name", "position", "jn", "player_name_2", "jn_2"]].sort_values(["position", "jn", "jn_2"])

,player_name,position,jn,player_name_2,jn_2
451,Marie-Philip Poulin,Center,29,Sarah Fillier,10
971,Erin Ambrose,Right Defense,23,Mich Zandee-Hart,28
1261,Erin Ambrose,Right Defense,23,Jamie Lee Rattray,47
1333,Mich Zandee-Hart,Right Defense,28,Renata Fast,14
1343,Mich Zandee-Hart,Right Defense,28,Renata Fast,14
20,Jamie Lee Rattray,Right Defense,47,Renata Fast,14
28,Jamie Lee Rattray,Right Defense,47,Renata Fast,14
1260,Jamie Lee Rattray,Right Defense,47,Renata Fast,14
1286,Jamie Lee Rattray,Right Defense,47,Renata Fast,14
